In [49]:
#!python3 -m spacy download en_core_web_sm
#!pip install geopy
import pandas as pd
import numpy as np
import NLP
from dataprep.clean import validate_country
import altair as alt
# instantiate one
import spacy 
nlp = spacy.load('en_core_web_sm')
#import geopy.geocoder
from geopy.geocoders import Nominatim

In [50]:
df = pd.read_csv('metaverse_tweets.csv')
df.head(3)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,Diep Ngoc,NaN,NaN,2021-08-22 09:53:23+00:00,162,129,1816,False,2022-02-27 23:59:46+00:00,@KingArthurrNFT My choice is #Elemon 😊🤩😍🥰\n#BS...,"['Elemon', 'BSC', 'NFT', 'GameFi', 'Metaverse'...",Twitter for Android,False
1,Duncs,"Melbourne, Australia",NFT's + Crypto + Metaverse + Outdoors + Goodtimes,2012-11-28 01:47:19+00:00,745,1363,2583,False,2022-02-27 23:59:33+00:00,Can't wait to hear this #Bapesclan \n\n#BapesM...,"['Bapesclan', 'BapesMetavestor', 'Metaverse']",Twitter Web App,False
2,Putik Gita,"Banyuwangi, Indonesia",just ordinary woman,2016-04-07 03:31:10+00:00,21,130,65,False,2022-02-27 23:59:26+00:00,@SuperStarPad @binance @Nuls good project.. ho...,NaN,Twitter for Android,False


## Cleaning Twitter Location Data

Removing #'s from the data

In [51]:
df.user_location.astype(str)
df = df[~df.user_location.str.contains("#", na=False)] 
df.head(4)                                  

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,Diep Ngoc,NaN,NaN,2021-08-22 09:53:23+00:00,162,129,1816,False,2022-02-27 23:59:46+00:00,@KingArthurrNFT My choice is #Elemon 😊🤩😍🥰\n#BS...,"['Elemon', 'BSC', 'NFT', 'GameFi', 'Metaverse'...",Twitter for Android,False
1,Duncs,"Melbourne, Australia",NFT's + Crypto + Metaverse + Outdoors + Goodtimes,2012-11-28 01:47:19+00:00,745,1363,2583,False,2022-02-27 23:59:33+00:00,Can't wait to hear this #Bapesclan \n\n#BapesM...,"['Bapesclan', 'BapesMetavestor', 'Metaverse']",Twitter Web App,False
2,Putik Gita,"Banyuwangi, Indonesia",just ordinary woman,2016-04-07 03:31:10+00:00,21,130,65,False,2022-02-27 23:59:26+00:00,@SuperStarPad @binance @Nuls good project.. ho...,NaN,Twitter for Android,False
3,🍑𝗔𝗕𝗗 𝗖𝗫 𝟏𝟎𝟎𝐊🍑,"Dhaka, Bangladesh",🍑🌱🍑𝐅𝐎𝐋𝐋𝐎𝐖 𝐌𝐄 🍑🌱🍑\n𝐈 𝐅𝐎𝐋𝐋𝐎𝐖 𝐁𝐀𝐂𝐊 𝐅𝐀𝐒𝐓🍑𝟏𝟎𝟎%𝐈𝐅𝐁🍑\...,2021-10-15 18:21:33+00:00,2355,3349,8550,False,2022-02-27 23:59:13+00:00,Starts Trading on Uniswap and XT Exchanges Feb...,NaN,Twitter Web App,False


Remove irelevant words

In [52]:
df["user_location"] = df["user_location"].astype(str)
print(df['user_location'].dtype)
df['extracted_user_location'] = df['user_location'].apply(lambda x: list(nlp(x).ents) if len(list(nlp(x).ents))>0 else np.nan) 
#return na value if list is empty

object


Using GeoPy API to get coordinates of locations 

In [53]:
# Ref: https://geopy.readthedocs.io/en/stable/#usage-with-pandas
geolocator = Nominatim(user_agent='my_email@myserver.com')

#Currently Have a weird Rate Limiter error not sure what to do 

geocode = RateLimiter(geolocator.geocode, min_delay_seconds=3, max_retries=5)
df["location"] = df["user_location"].progress_apply(geocode, language="en") # Some locations are in hindi, chinese. Language=’en’ returns location in english
df["coordinates"] = df["location"].apply(lambda loc: tuple(loc.point) if loc else None)
df["state"] = df["location"].apply(lambda loc: loc[0].split(",")[0] if loc else None)
df["country"] = df["location"].apply(lambda loc: loc[0].split(",")[-1] if loc else None)
df

NameError: name 'RateLimiter' is not defined